<a href="https://colab.research.google.com/github/Kamesuta/whisper-openai/blob/master/WhisperDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAIのWhisperを使って、文字起こしをしよう！
Whisperでの書き起こしテンプレートです。  
手順に沿って実行していけば、音声ファイルや動画ファイルの書き起こしができます。  

## スペシャルサンクス
以下のデモをベースに改変して作成しました。  
https://github.com/fastforwardlabs/whisper-openai  
<a href="https://colab.research.google.com/github/fastforwardlabs/whisper-openai/blob/master/WhisperDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 使い方！
1. 書き起こす動画/音声をアップロードします。  
![書き起こす動画/音声をアップ](https://i.gyazo.com/cddc2a1f0e34eb3d5a2cc3b1093f1a05.png)
2. ↓にある[▶]ボタンをポチポチ押していきます。

## インストールとインポート
以下のコマンドは、音声スニペットを録音し、Whisperモデルを使用して音声からテキストに書き起こすために必要なPythonパッケージをインストールします。

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install notebook

動画のエンコードに必要な必要なソフトをインストールします。

In [ ]:
!apt install ffmpeg
!apt-get install libportaudio2

In [12]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

if DEVICE == "cuda":
  print("使用モード(CPU/GPU): GPU")
else:
  print("使用モード(CPU/GPU): CPU")

使用モード(CPU/GPU): GPU


### 音声ファイルに変換する
Whisperに読み込ませられる音声ファイルの形式に変換します。  
アップロードした音声ファイル名をテキストボックスに入力して、「変換開始」ボタンを押してください。

### OBSなどで録音した場合
OBSなどで、自分のマイクと、Discord音声の音声を別トラックで録画している場合などは、1つのトラックに結合する必要があります。  
その場合は「変換開始 (結合)」ボタンを押してください。  


In [13]:
import ipywidgets as widgets
from IPython.display import display

# テキストボックスを作成
video_file_input = widgets.Text(
    value='',
    placeholder='動画/音声ファイル名を入力してください (2024-01-01_01-01-01.mp4)',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

# ボタンを作成
button = widgets.Button(description='変換開始')
button_mix = widgets.Button(description='変換開始 (結合)')

def on_button_clicked(b):
    video_file_name = video_file_input.value
    # ffmpegコマンドを実行
    !ffmpeg -i "{video_file_name}" "voice.mp3"

def on_button_clicked_mix(b):
    video_file_name = video_file_input.value
    # ffmpegコマンドを実行 (結合)
    !ffmpeg -i "{video_file_name}" -map 0:1 -map 0:2 -filter_complex amix=inputs=3:duration=first:dropout_transition=3 "voice.mp3"

button.on_click(on_button_clicked)

# テキストボックスとボタンを表示
display(video_file_input, button, button_mix)


Text(value='', layout=Layout(width='50%'), placeholder='動画/音声ファイル名を入力してください (2024-01-01_01-01-01.mp4)')

Button(description='変換開始', style=ButtonStyle())

Button(description='変換開始 (結合)', style=ButtonStyle())

## 言語を選択

日本語の書き起こしをするためには「japanese」のままにしてください

In [14]:
language_options = whisper.tokenizer.TO_LANGUAGE_CODE
language_list = list(language_options.keys())

lang_dropdown = widgets.Dropdown(options=language_list, value='japanese')
output = widgets.Output()
display(lang_dropdown)

Dropdown(index=7, options=('english', 'chinese', 'german', 'spanish', 'russian', 'korean', 'french', 'japanese…

## 通訳を行うか
単純に書き起こしだけを行うためには「transcribe」のままにしてください。  
書き起こし+通訳もしたい場合は「translate」にしてください。

In [15]:
task_dropdown = widgets.Dropdown(options=['transcribe', 'translate'], value='transcribe')
output = widgets.Output()
display(task_dropdown)

Dropdown(options=('transcribe', 'translate'), value='transcribe')

## Whisperのモデルを読み込む

モデルをダウンロードして読み込みます

In [ ]:
if lang_dropdown.value == "english":
  model = whisper.load_model("base.en")
else:
  model = whisper.load_model("base")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

## オプションを構築します  
設定が↓に表示されます

In [ ]:
options = whisper.DecodingOptions(language=lang_dropdown.value, task=task_dropdown.value, without_timestamps=True)
options

## Whisperで書き起こしを開始！

あとは音声をWhisperに送り込んで書き起こしを開始します  
書き起こし結果が以下に表示されます。

In [ ]:
result = model.transcribe("voice.mp3", verbose=True, language=lang_dropdown.value, task=task_dropdown.value, without_timestamps=False)